# Generate Metadata

This will take a logfile and split it to orbits, then read the code between the orbits and provide data on the satelite's health detections etc.

In [1]:
# Import libraries
import raadpy as rp
import pandas as pd

# Load the data filenames
raw_dir = './'
filenames = rp.get_filenames(raw_dir)
print(filenames)

# To have interactive plots in the jupyter notebook
%matplotlib notebook

Warning! Cupy not found! GPU accelleration is not available

{'log': 'RAAD-0-log.txt'}


In [2]:
# Now we load the log file and find it's timestamps
log     = rp.log_expand(filename=raw_dir+filenames['log'])      # Load the logfile
log     = rp.log_with_timestamp(log,reorder=False)              # Find the timestamp that each command finished executing

Found weird command
 {'command': 'SE0>script delayuntil 1# Start Of Script 1 Jun 2022 12:38:25.005\n', 'output': [], 'index': 31072}
Found weird command
 {'command': 'SE0>rtc read\n', 'output': ['curr rtc time 165589447script delayuntil 1655955900\n', 'OK\n'], 'index': 63743}
Index Error in command
 {'command': 'SE0>rtc read\n', 'output': ['script delayuntil 1657003500\n', 'OK\n'], 'index': 95349}
Found weird command
 {'command': 'SE0>script delayuntil 1657246500# Start Of Script 9 July 2022 0:00:00\n', 'output': [], 'index': 106148}
Found weird command
 {'command': 'SE0>rtc read\n', 'output': ['curr rtc ti# Start Of Script 20 July 2022 0:00:00\n'], 'index': 133934}
Found weird command
 {'command': 'SE0>rtc read\n', 'output': ['curr rtc time# Start Of Script 22 July 2022 0:00:00\n'], 'index': 140750}
Found weird command
 {'command': 'SE0>script delayuntil 165946script delayuntil 1659465900\n', 'output': ['OK\n'], 'index': 171984}
Found weird command
 {'command': 'SE0>script delayuntil 

In [3]:
# Collect the timestamps for when an orbit started and ended
start_timestamps_log, start_idx    = rp.collect_time_cmd(log,rp.PWR_ON_CMD,include_idx=True).T
end_timestamps_log, end_idx      = rp.collect_time_cmd(log,rp.PWR_OFF_CMD,include_idx=True).T

# Match the two arrays from the back

end_timestamps_log, start_timestamps_log, start_idx, end_idx = rp.match(end_timestamps_log,start_timestamps_log,start_idx.astype('int'),end_idx.astype('int'))

In [11]:
# Empty array for metadata
metadata = []
# For every power on and off cycle get the metadata
for s,e in zip(start_idx,end_idx):
    sub_log = log[s:e]
    metadata.append(rp.log_metadata(sub_log))

# Single complete dictionary
meta_complete = {}

# Now convert the metadata to a signle dictionary
meta_complete = {i: [cycle[i] for cycle in metadata] for i in metadata[0]}
for key in meta_complete:
    if type(meta_complete[key][0]) is dict:
        meta_complete[key] = {i: [cycle[i] for cycle in meta_complete[key]] for i in meta_complete[key][0]}

In [12]:
# Save the data to a csv file
# Flatten the dictionary
[flat_dict] = pd.json_normalize(meta_complete, sep='.').to_dict(orient='records')
# Create a pandas dataframe and save
pd.DataFrame.from_dict(flat_dict).to_csv (r'orbit_meta.csv', index = True, header=True)